In [1]:
!nvidia-smi

Mon Jan 23 13:51:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 470.74       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    Off  | 00000000:1A:00.0 Off |                  N/A |
| 40%   54C    P2    64W / 280W |   2966MiB / 24220MiB |     11%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import transformers

transformers.__version__

'2.3.0'

In [3]:
import pyterrier as pt
# pt.init(tqdm='notebook')
pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [4]:
import faiss
faiss.get_num_gpus()

1

In [5]:
import pandas as pd

qrels2019 = pt.get_dataset("trec-deep-learning-passages").get_qrels('test-2019')
topics2019 = pt.get_dataset("trec-deep-learning-passages").get_topics('test-2019')
topics2019 = topics2019.merge(qrels2019[qrels2019["label"] > 0][["qid"]].drop_duplicates())



topics2020 = pt.get_dataset("trec-deep-learning-passages").get_topics('test-2020')
qrels2020 = pt.get_dataset(  "trec-deep-learning-passages").get_qrels('test-2020')
topics2020 = topics2020.merge(qrels2020[qrels2020["label"] > 0][["qid"]].drop_duplicates())



# ANCE-PRF

In [6]:

import pyterrier_ance
# build the ANCE retriever
anceRetr = pyterrier_ance.ANCERetrieval("/nfs/craigm/Passage ANCE(FirstP) Checkpoint", 
                                        "/nfs/craigm/ance_msmarco_passages/",num_results=2000)
# load the ANCE-PRF k=3 checkpoint
anceprf = pyterrier_ance.ANCEPRF(checkpoint_path="./k3_checkpoint")
# # retrieval using the refined query rerpesentation using ANCE
anceRetr_prf = pyterrier_ance.ANCERetrieval( "/nfs/craigm/Passage ANCE(FirstP) Checkpoint","/nfs/craigm/ance_msmarco_passages/", query_encoded=True,num_results=2000,index_on_gpu=False)



Loading checkpoint /nfs/craigm/Passage ANCE(FirstP) Checkpoint
Using mean: False
Loading shard metadata


Loading shards: 100%|██████████| 18/18 [00:19<00:00,  1.10s/shard]


Loading checkpoint ./k3_checkpoint
Using mean: False
Loading checkpoint /nfs/craigm/Passage ANCE(FirstP) Checkpoint
Using mean: False
Loading shard metadata


Loading shards: 100%|██████████| 18/18 [00:18<00:00,  1.05s/shard]


In [13]:
pipe_anceprf =anceRetr%10>>pt.text.get_text(pt.get_dataset('irds:msmarco-passage'), 'text')>>anceprf>>anceRetr_prf

# Vector-PRF

In [8]:
qencoder= pyterrier_ance.ANCEQueryEncoder(checkpoint_path="/nfs/craigm/Passage ANCE(FirstP) Checkpoint")
dencoder = pyterrier_ance.ANCETextEncoder(checkpoint_path="/nfs/craigm/Passage ANCE(FirstP) Checkpoint")
vectorprf = pyterrier_ance.PRF_VectorAverage()

Loading checkpoint /nfs/craigm/Passage ANCE(FirstP) Checkpoint
Using mean: False
Loading checkpoint /nfs/craigm/Passage ANCE(FirstP) Checkpoint
Using mean: False


In [11]:
pipe_vectorprf = (anceRetr%10
        >>qencoder
        >>pt.text.get_text(pt.get_dataset('irds:msmarco-passage'), 'text')
        >>dencoder>>pyterrier_ance.PRF_VectorAverage(k=3,prf_type = "vector_prf")
        >>anceRetr_prf
        
       )

In [12]:
pipe_rocchioprf = (anceRetr%10
        >>qencoder
        >>pt.text.get_text(pt.get_dataset('irds:msmarco-passage'), 'text')
        >>dencoder>>pyterrier_ance.PRF_VectorAverage(k=3,alpha=0.4,beta=0.6,prf_type = "rocchio_prf")
        >>anceRetr_prf
        
       )

In [14]:

from pyterrier.measures import *
res = pt.Experiment(
    [anceRetr, pipe_anceprf, pipe_vectorprf, pipe_rocchioprf
    ],
    topics2019,
    qrels2019,
    save_dir="./",
    eval_metrics=[AP(rel=2)@1000, nDCG@10,R(rel=2)@1000,"mrt"],
#     baseline=0,
    names=["ance.dl19","ance-prf.dl19","vector-prf.dl19",'rocchio-prf.dl19'], verbose=True, batch_size=10,
)

res

pt.Experiment:  10%|█         | 2/20 [00:00<00:06,  2.82batches/s]

***** inference of 10 queries *****


Inferencing: 1it [00:00, 21.89it/s]


Not running in distributed mode
***** faiss search for 10 queries on 18 shards *****



100%|██████████| 18/18 [00:40<00:00,  2.25s/shard]
Inferencing: 1it [00:00, 21.52it/s]


...

Not running in distributed mode


Inferencing: 1it [00:00,  8.82it/s]


Not running in distributed mode
***** faiss search for 3 queries on 18 shards *****



pt.Experiment:  40%|████      | 8/20 [06:05<09:08, 45.73s/batches]


,name,AP(rel=2)@1000,nDCG@10,R(rel=2)@1000,mrt
0,ance.dl19,0.371512,0.653703,0.757131,0.00000
1,ance-prf.dl19,0.426855,0.677552,0.793970,0.00000
2,vector-prf.dl19,0.428616,0.662689,0.774935,8435.19527
3,rocchio-prf.dl19,0.421635,0.659355,0.774879,0.00000
